## C S 329E HW 5

# Crossvalidation and hyperparameter selection

## Srijeeta Biswas, Isabella Joseph

For this week's homework we are going to explore the cross validation testing methodology and applying that to get error estimates on the two algorithms we've used so far:
  - Linear Regression
  - Decision Tree classification
  

In [1]:
# import the libraries! If you want to add things here for visualization, please do, 
# but only use sklearn when prompted
import pandas as pd
import numpy as np
from sklearn import tree 
from sklearn.datasets import load_iris
from sklearn.datasets import load_diabetes
import sklearn
from sklearn.metrics import mean_absolute_error

In [2]:
!pip install --upgrade pip

Requirement already up-to-date: pip in /Users/srijeetabiswas/opt/anaconda3/lib/python3.7/site-packages (20.2.3)


In [3]:
!pip install --upgrade sklearn

Requirement already up-to-date: sklearn in /Users/srijeetabiswas/opt/anaconda3/lib/python3.7/site-packages (0.0)


# Part 1 - Calculate Generalized Error on Linear Regression with k-fold Cross Validation

## Q1.1 Load in the diabetes data set as a pandas dataframe and series.  
Documentation is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html).  Name your features dataframe (the independent variables) `df_X` and your target (the dependent variable) series `s_y`

In [4]:
input_data =sklearn.datasets.load_diabetes()
x = input_data['data']
y = input_data['target']
features = input_data['feature_names']

df_X = pd.DataFrame(x, columns = features)
# print(input_data)
s_y = pd.DataFrame(y, columns = ['target'])
a = int(len(df_X)/5)


## Q1.2 Define a function that creates a linear least squares regression model 
This function should take in two parameters, `df_X`, and `s_y` and return the least squares regression model, $\hat{\beta}$ (using the notation from the ESLII book chapter 3 and HW2).  You can not use any libraries outside of pandas and numpy. 

In [5]:
def get_linear_regression_model( df_X, s_y ):
    df_X['constant'] = np.ones((df_X.shape[0]))
    beta = np.linalg.lstsq(df_X, s_y)
    return(beta)


## Q1.3 Define a function that partitions the dataframe and series data into dictionaries
This function should take in three parameters, `df_X`, `s_y`, and `k`, and returns a tuple of two dictionaries.
In both dictionaries the key is the `k` value (an integer from one to `k` inclusive).
The first dictionary will have the dataframe of the training data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable).
The second dictionary will have the series of the target data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable). Please note the targets _must match_ the same rows as the dataframe at this index, e.g, the length of the kth partition is the same for the dataframe and series.

Call that function with $k=5$ and create the dictionaries `dict_k_df_X` and `dict_k_s_y`. Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

In [6]:
def partition_data( df_X, s_y, k ):
    dict_df_X = {}
    dict_s_y = {}
    k_frac = 1/k
    rows_X = len(df_X)
    section_X = int(rows_X*k_frac)
    rows_y = len(s_y)
    section_y = int(rows_y*k_frac)
    start_X = 0
    start_y = 0
    

    
    df_X = df_X.reset_index(drop = True)
    s_y = s_y.reset_index(drop = True)
    
    new_df = pd.concat([df_X,s_y],axis = 1)
    df_shuffling = new_df.sample(frac=1)


    s_y_shuffling = df_shuffling['target']
    df_X_shuffling = df_shuffling.drop(df_shuffling.columns[-1],axis=1)
    
    for i in range(1,k+1,1):

    
        end_X = i*section_X
        end_y = i*section_y
        dict_df_X[i] = df_X_shuffling.iloc[start_X:end_X]
        dict_s_y[i] = s_y_shuffling.iloc[start_y:end_y]
        start_X = end_X
        start_y = end_y
    return(dict_df_X, dict_s_y)

In [7]:
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5 )


## Q1.4 Define a function that calculates a regression metric
This function should accept two series of equal length $n$ numpy arrays, `s_y`, and `s_y_hat`. The metric it should calculate is the mean absolute error, $MAE = \sum\limits_{i=1}^n\frac{|{s\_y_i - {s\_y\_hat}_i}|}{n}$ 

Test your function by using the vectors:
```
x = np.array([1,2,3])
y = np.array([2,2,3])
```


In [8]:
from sklearn.metrics import mean_absolute_error

In [9]:
## can we use sklearn 
# if we cannot- use for loop 

def get_mae( s_y, s_y_hat):
    result = mean_absolute_error(s_y, s_y_hat)
    return(result)

In [10]:
# Test it 
x = np.array([1,2,3])
y = np.array([2,2,3])
get_mae(x,y)

0.3333333333333333

## Q1.5 Calculate the $MAE$ for each fold
For each fold in your dictionaries, calculate the $MAE$.  Use the partition number key as the test set, and all other partitions as the train set. 

Print the min, max, and mean $MAE$ of your 5 folds. 

In [11]:
mae = np.array([])
for k in dict_k_df_X.keys():
    # iterating through the folds
    df_X = dict_k_df_X[k]
    s_y = dict_k_s_y[k]
    # getting beta for each fold
    
#     print(len(df_X))
#     print(len(s_y))
    
    beta = get_linear_regression_model(df_X, s_y)
    beta = beta[0]
    df_X = np.array(df_X)
    
    # predicted y
    s_y_hat = np.matmul(df_X, beta)
    
    # computing the error 
    mae = np.append( mae, get_mae(s_y,s_y_hat) )

print('Min MAE: ' + str(min(mae)))
print('Max MAE: ' + str(max(mae)))
print('Mean MAE: ' + str(np.mean(mae)))

Min MAE: 37.58963477044445
Max MAE: 44.72357034203996
Mean MAE: 41.29246577147571


/Users/srijeetabiswas/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/srijeetabiswas/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/srijeetabiswas/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: Futur

# Part 2 - Find the best hyperparameter to use in a Decision Tree 

## Q2.1 Load the iris data in as a pandas dataframe and a series
Documentation is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html). Name your features dataframe (the independent variables) `df_X` and your classification target (the dependent variable) series `s_y`

In [12]:
in_data = sklearn.datasets.load_iris()
x2 = in_data['data']
y2 = in_data['target']
features2 = in_data['feature_names']

df_X = pd.DataFrame(x2, columns = features2)
s_y = pd.DataFrame(y2, columns = ['target'])


## Q2.2 Partition `df_X` and `s_y` into $5$ partitions of roughly equal size
Make 2 dictionaries, with the key of each dictionary the fold number.  The value of the dictionary `dict_k_df_X` is the $k^{th}$ partition of the data, and the value of the dictionary `dict_k_s_y` is the corresponding $k^{th}$ target classification.  Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

In [13]:
rows = len(df_X)
# print(df_X)
dict_k_df_X, dict_k_s_y = partition_data(df_X, s_y, 5)
total = 0
# print(s_y)
for key in dict_k_df_X:
    total+=  len(dict_k_df_X[key])
    print("number of rows in df_X: "+ str(len(dict_k_df_X[key])))
    print("number of rows in s_y: "+ str(len(dict_k_s_y[key])))
    print()
print("Total sum for df_X and s_y: "+ str(total))

number of rows in df_X: 30
number of rows in s_y: 30

number of rows in df_X: 30
number of rows in s_y: 30

number of rows in df_X: 30
number of rows in s_y: 30

number of rows in df_X: 30
number of rows in s_y: 30

number of rows in df_X: 30
number of rows in s_y: 30

Total sum for df_X and s_y: 150


## Q2.3 Define a function that calculates accuracy
The function should accept two series and compare each element for equality.  The accuracy is the number of equal elements divided by the total number of elements.

Test your accuracy function by calling it with the `s_y` loaded from the iris data set and an array of the same length containing all $1$ values. 

In [14]:
def get_acc( s_1, s_2 ):
    correct= 0
    total = len(s_1)
    for r in range(total):
        if isinstance(s_1, pd.Series):
            if s_1.iloc[r] == s_2[r]:
                correct += 1
        else:
            if s_1.iloc[r][0] == s_2[r]:
                correct += 1
    return(correct/total)

In [15]:
get_acc(s_y,np.ones(len(s_y)))

0.3333333333333333

## Q2.4 Using Nested Cross validation, find the best hyperparameter
Use the [Decision Tree Classifier](https://scikit-learn.org/stable/modules/tree.html#classification) class to build a decision tree inside of a 5-fold cross validation loop.  The partitions you already created in 2.2 will be the outer loop.  In the inside loop you should use 4-fold cross validation (so you don't have to partition _again_) to find the best value for `min_impurity_decrease`.  Use the Gini Index as your impurity measure. 
    Calculate the mean accuracy across the 4 folds of your inner loop for all the candidate `min_impurity_decrease` values, and print the value.  Use the array `np.array([0.1,0.25,0.3,0.4])` as the candidates for the best hyperparameter. If there is a tie (two `min_impurity_decrease` values give the same highest accuracy), choose the lowest `min_impurity_decrease` value. 

For each inner loop, select the best `min_impurity_decrease` and train the outer fold training data on using that value. 

For each inner loop, your output should look something like this:
```
Testing 0.10 min impurity decrease
	Average accuracy over 4 folds is 0.95
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.86
Testing 0.30 min impurity decrease
	Average accuracy over 4 folds is 0.63
Testing 0.40 min impurity decrease
	Average accuracy over 4 folds is 0.27

Best min impurity decrease is 0.1

```

In [16]:
from copy import deepcopy

In [17]:

possible_min_impurity_decrease = np.array([0.1,0.25,0.3,0.4])
dict_k_df_X_inner = deepcopy(dict_k_df_X)
dict_k_s_y_inner = deepcopy(dict_k_s_y)
accuracy = {}
# Outer loop
outer_acc = []

for final in possible_min_impurity_decrease:
    accuracy[final] = []

for k in dict_k_df_X.keys():
    test_set_X = dict_k_df_X[k]
    test_set_Y = dict_k_s_y[k]
    
        
    del dict_k_df_X_inner[k]
    del dict_k_s_y_inner[k]
    
    
    x_training = pd.DataFrame()
    y_training = pd.DataFrame()
    
    for element in dict_k_df_X_inner.keys():
        x_training = pd.concat([x_training, dict_k_df_X_inner[element]],axis=0)
        y_training = pd.concat([y_training, dict_k_s_y_inner[element]],axis=0)

    for pos_min_impurity in possible_min_impurity_decrease:
        
        
        
        for j in dict_k_df_X.keys():
            if j == k:
                train2_x = pd.DataFrame()
                train2_y = pd.DataFrame()
                
                test2_x = dict_k_df_X[j]
                test2_y = dict_k_s_y[j]
                
                for element in range(1, len(dict_k_df_X) + 1):
                    if element != j and element != k:
                        train2_x = pd.concat([train2_x, dict_k_df_X[element]], axis = 0)
                        train2_y = pd.concat([train2_y, dict_k_s_y[element]], axis = 0)
                        

                clf = tree.DecisionTreeClassifier(criterion = 'gini', min_impurity_decrease= pos_min_impurity)
                clf.fit(train2_x, train2_y)
                
                
                predicted_y = clf.predict(test2_x)


                inner_acc = get_acc(test2_y,predicted_y)
                accuracy[pos_min_impurity].append(inner_acc)
    dict_k_df_X_inner[k] = dict_k_df_X[k]
    dict_k_s_y_inner[k] = dict_k_s_y[k]

                
    all_avg = {}
    for pos_min_impurity in possible_min_impurity_decrease:
        all_avg[pos_min_impurity] = (sum(accuracy[pos_min_impurity]))/len(accuracy[pos_min_impurity])
        print('Testing '+ str(pos_min_impurity) + ' min impurity decrease')
        print('\tAverage accuracy over 4 folds is '+str((round(all_avg[pos_min_impurity],2))))
    print()
 
    
    
    max_score = max(all_avg.values())
    final_hyperparameter_list = []
    for keyss in all_avg:
        if all_avg[keyss] == max_score:
            final_hyperparameter_list.append(keyss)
    final_hyperparameter = min(final_hyperparameter_list)
    
    print('Best min impurity decrease is ' + str(final_hyperparameter))
    print()
    print()
    clf_outer = tree.DecisionTreeClassifier(criterion = 'gini', min_impurity_decrease= final_hyperparameter)
    clf_outer.fit(x_training, y_training)
    
    
    outer_pred = clf_outer.predict(test_set_X)
    
    outer_accuracy = get_acc(test_set_Y, outer_pred)
    
    
    outer_acc.append(outer_accuracy)











Testing 0.1 min impurity decrease
	Average accuracy over 4 folds is 0.93
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.93
Testing 0.3 min impurity decrease
	Average accuracy over 4 folds is 0.63
Testing 0.4 min impurity decrease
	Average accuracy over 4 folds is 0.23

Best min impurity decrease is 0.1


Testing 0.1 min impurity decrease
	Average accuracy over 4 folds is 0.97
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.75
Testing 0.3 min impurity decrease
	Average accuracy over 4 folds is 0.6
Testing 0.4 min impurity decrease
	Average accuracy over 4 folds is 0.23

Best min impurity decrease is 0.1


Testing 0.1 min impurity decrease
	Average accuracy over 4 folds is 0.94
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.8
Testing 0.3 min impurity decrease
	Average accuracy over 4 folds is 0.6
Testing 0.4 min impurity decrease
	Average accuracy over 4 folds is 0.24

Best min impurity decrease is 0.1


Testing 0.1 m

## Q2.5 Show the generalized performance of the classifier 
Show the generalized performance of the classifier by printing the min, max, and mean accuracy of the outer fold test sets.

In [18]:
minimum = round(min(outer_acc),2)
maximum = round(max(outer_acc),2)
average = round(np.mean(outer_acc),2)
print('Min accuracy of outer fold test sets: '+ str(minimum))
print('Max accuracy of outer fold test sets: ' + str(maximum))
print('Mean accuracy of outer fold test sets: ' + str(average))

Min accuracy of outer fold test sets: 0.9
Max accuracy of outer fold test sets: 1.0
Mean accuracy of outer fold test sets: 0.94
